In [ ]:
import os
import matplotlib.pyplot as plt

from lsmcpp.planners import *
from lsmcpp.local_search import *
from lsmcpp.turn_minimization.TMSTCStar import TMSTCStar_planner

### load the floor_small MCPP instance

In [ ]:
name = "floor_small"

# Unzip data files
if not os.path.exists(f"data/gridmaps/{name}.map"):
    import subprocess
    subprocess.call(["unzip", "-o", "data/data.zip", "-d", "data"])

mcpp = MCPP.read_instance(os.path.join("data", "instances", f"{name}.mcpp"))

fig, ax = plt.subplots()
mcpp.draw(ax)

### Run LS-MCPP with MFC as initial solution and TMSTC*

In [ ]:
init_sol = MFC_planner(mcpp)
ls_mcpp_planner = LocalSearchMCPP(mcpp, init_sol, PrioType.CompositeHeur, PoolType.VertexEdgewise)
M = 2000
sol_opt, rt = ls_mcpp_planner.run(
    M=M,
    S=int(M // 20),
    alpha=np.exp(np.log(0.2) / M),
    gamma=0.01,
    sample_type=SampleType.RouletteWheel
)

tmstcstar_sol = TMSTCStar_planner(mcpp)


In [ ]:
colors = ["r", "b", "k", "c"]

def draw_solution(ax, sol):
    for i, r in enumerate(mcpp.R):
        r = mcpp.legacy_vertex(r)
        ax.plot(r[0], r[1], "*k", ms=8, mfc=colors[i])

    for i, r in enumerate(mcpp.R):
        r = mcpp.legacy_vertex(r)
        ax.plot(r[0], r[1], "*k", ms=8, mfc=colors[i])

    for v in mcpp.static_obstacles:
        px, py = (v[0] - 0.5) / 2, (v[1] - 0.5) / 2
        ax.plot(px, py, "ks", ms=8)

    for i in range(mcpp.k):
        pi = sol.Pi[i]
        for j in range(len(pi)-1):
            ax.plot([pi[j][0], pi[j+1][0]], [pi[j][1], pi[j+1][1]], color=f'{colors[i]}', linewidth=4, alpha=0.6)
    ax.axis("equal")

fig, axs = plt.subplots(1, 2)

draw_solution(axs[0], sol_opt)
draw_solution(axs[1], tmstcstar_sol)

